In [ ]:
import umap
from spacy.lang.de import German
from tqdm import tqdm
from spacy.lang.de.stop_words import STOP_WORDS
import pandas as pd
import json
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from wordcloud import WordCloud
import seaborn as sns
from gensim.models import Word2Vec
import numpy as np
from sklearn import metrics
from sklearn.decomposition import PCA

### Reading

In [ ]:
f = open("cleared_data.txt", "r", encoding="utf-8")
cleared_data_law = f.readlines()
f.close()

In [ ]:
cleared_data = []
for var in cleared_data_law:
    temp = []
    for j in list(var.split(" ")):
        temp.append(j)
    cleared_data.append(temp)

### TF-IDF

In [ ]:
vectorizer = TfidfVectorizer(max_features = 2000)
X = vectorizer.fit_transform(cleared_dataa)

In [ ]:
from sklearn.preprocessing import normalize
tf_idf_norm = normalize(X)
tf_idf_array = tf_idf_norm.toarray()

In [ ]:
Sum_of_squared_distances = []
K = range(2,10)
for k in K:
    print(k)
    km = KMeans(n_clusters=k, max_iter=50, n_init=1)
    km = km.fit(X)
    Sum_of_squared_distances.append(km.inertia_)
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
true_k = 4
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)
labels=model.labels_
law_cl=pd.DataFrame(list(zip(list(category),labels)),columns=['category','cluster'])
print(law_cl.sort_values(by=['cluster']))

In [ ]:
true_k = 6
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=200, n_init=1)
model.fit(X)
labels=model.labels_
law_cl=pd.DataFrame(list(zip(list(category),labels)),columns=['category','cluster'])
print(law_cl.sort_values(by=['cluster']))

In [ ]:
words = modell.wv.vocab.keys()
feature_names = list(words)

In [ ]:
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print ("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print (' %s' % terms[ind]),
    print()

In [ ]:
result={'cluster':labels,'law':new_data_list}
result=pd.DataFrame(result)
for k in range(0,true_k):
    s=result[result.cluster==k]
    text=s['law'].str.cat(sep=' ')
    text=text.lower()
    text=' '.join([word for word in text.split()])
    wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(text)
    print('Cluster: {}'.format(k))
    print('Titles')
    titles=law_cl[law_cl.cluster==k]['category']         
    print(titles.to_string(index=False))
    plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()

In [ ]:
result={'cluster':labels,'law':new_data_list}
result=pd.DataFrame(result)
for k in range(0,true_k):
    s=result[result.cluster==k]
    text=s['law'].str.cat(sep=' ')
    text=text.lower()
    text=' '.join([word for word in text.split()])
    wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(text)
    print('Cluster: {}'.format(k))
    print('Titles')
    titles=law_cl[law_cl.cluster==k]['category']         
    print(titles.to_string(index=False))
    plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()

In [ ]:
from sklearn.metrics import pairwise_distances_argmin

def find_clusters(X, n_clusters, rseed=2):
    # 1. Randomly choose clusters
    rng = np.random.RandomState(rseed)
    i = rng.permutation(X.shape[0])[:n_clusters]
    centers = X[i]
    
    while True:
        # 2a. Assign labels based on closest center
        labels = pairwise_distances_argmin(X, centers)
        
        # 2b. Find new centers from means of points
        new_centers = np.array([X[labels == i].mean(0)
                                for i in range(n_clusters)])
        
        # 2c. Check for convergence
        if np.all(centers == new_centers):
            break
        centers = new_centers
    
    return centers, labels

centers, labelss = find_clusters(X, 4)
plt.scatter(X[:, 0], X[:, 1], c=labelss,
            s=50, cmap='viridis');

In [ ]:
asdf = vectorizer.vocabulary_

In [ ]:
y = []
for asd in X:
    y.append(asd)

In [ ]:
random_state = 0
# reduce the features to 2D
pca = PCA(n_components=2, random_state=random_state)
reduced_features = pca.fit_transform(tf_idf_array)

# reduce the cluster centers to 2D
reduced_cluster_centers = pca.transform(model.cluster_centers_)

plt.scatter(reduced_features[:,0], reduced_features[:,1], c=model.predict(X))
plt.scatter(reduced_cluster_centers[:, 0], reduced_cluster_centers[:,1], marker='x', s=150, c='b')